In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("phase2_baby_all.csv")

In [6]:
df

,group_id,sentence,sentiment,uuid
0,1,I am primarily breastfeeding but I thought thi...,neg,1114646400B000056OUGA3FFDK09UJS1TD
1,1,I discovered Medela Microwave Steam Cleaning B...,neu,1114646400B000056OUGA3FFDK09UJS1TD
2,1,I feel badly for the waste of money but the ba...,neu,1114646400B000056OUGA3FFDK09UJS1TD
3,1,And not space consuming.,pos,1114646400B000056OUGA3FFDK09UJS1TD
4,1,Plus you can use them during travel at the off...,pos,1114646400B000056OUGA3FFDK09UJS1TD
5,1,Much more cost efficient and useful!,pos,1114646400B000056OUGA3FFDK09UJS1TD
6,1,This monitor is decent for the price I guess.,pos,1049587200B00005TNIHA1B23P1EVL3EGI
7,1,But I think I would have rather spent a little...,neg,1049587200B00005TNIHA1B23P1EVL3EGI
8,1,The pro to the monitor is that it picks up the...,pos,1049587200B00005TNIHA1B23P1EVL3EGI
9,1,But the problem is that there's so much static...,neg,1049587200B00005TNIHA1B23P1EVL3EGI


In [7]:
data = df[['sentiment', 'sentence']].rename(columns={"sentiment":"label", "sentence":"text"})
 
data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [13]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path


corpus = NLPTaskDataFetcher.load_classification_corpus(Path(''), test_file = 'test.csv', dev_file = 'dev.csv', train_file = 'train.csv')#.downsample(0.01)

word_embeddings = [WordEmbeddings('glove'), 
                   FlairEmbeddings('news-forward-fast', use_cache = True), 
                   FlairEmbeddings('news-backward-fast', use_cache = True)]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, 
                                            hidden_size = 16, 
                                            reproject_words = True, 
                                            reproject_words_dimension = 256)

classifier = TextClassifier(document_embeddings, label_dictionary = corpus.make_label_dictionary(), multi_label = False)
trainer = ModelTrainer(classifier, corpus)

2019-05-10 09:04:55,767 Reading data from .
2019-05-10 09:04:55,769 Train: train.csv
2019-05-10 09:04:55,770 Dev: dev.csv
2019-05-10 09:04:55,771 Test: test.csv


In [12]:
trainer.train('./', max_epochs = 5)

2019-05-10 09:03:32,332 ----------------------------------------------------------------------------------------------------
2019-05-10 09:03:32,333 Evaluation method: MICRO_F1_SCORE
2019-05-10 09:03:32,335 ----------------------------------------------------------------------------------------------------
2019-05-10 09:03:34,957 epoch 1 - iter 0/392 - loss 0.04563719
2019-05-10 09:04:27,153 ----------------------------------------------------------------------------------------------------
2019-05-10 09:04:27,154 Exiting from training early.
2019-05-10 09:04:27,155 Saving model ...
2019-05-10 09:04:30,378 Done.
2019-05-10 09:04:30,383 ----------------------------------------------------------------------------------------------------
2019-05-10 09:04:30,384 Testing using best model ...


KeyboardInterrupt: 